In [1]:
import numpy as np
import time
from typing import List, Tuple

# --- Constantes do Projeto (Baseado no VHDL) ---
C_DATA_WIDTH = 8
C_VECTOR_SIZE = 4
# Limites de 8 bits com sinal (np.int8)
MAX_VAL = np.int8(127)
MIN_VAL = np.int8(-128)

def saturate_8bit(value: int) -> np.int8:
    """
    Implementa a lógica de Saturação (Clipping) para 8 bits com sinal.
    Se o resultado exceder 127, retorna 127.
    Se o resultado for menor que -128, retorna -128.
    """
    if value > MAX_VAL:
        return MAX_VAL
    if value < MIN_VAL:
        return MIN_VAL
    # Retorna o valor original, mas forçado a np.int8
    return np.int8(value)

def alu_vector_op(vector_a: List[int], vector_b: List[int], alu_op_sel: int) -> Tuple[List[np.int8], int]:
    """
    Simula o Somador/Subtrator Vetorial, operando em 4 elementos de 8 bits em paralelo.
    
    Args:
        vector_a (List[int]): 4 elementos de 8 bits (como lista/array).
        vector_b (List[int]): 4 elementos de 8 bits (como lista/array).
        alu_op_sel (int): 0 para SOMA, 1 para SUBTRAÇÃO.
        
    Returns:
        Tuple[List[np.int8], int]: O vetor de resultado (4 elementos saturados) 
                                   e o código da operação (0 ou 1).
    """
    
    result_vector = []
    
    # 1. Processamento Paralelo Elemento a Elemento
    for a_elem, b_elem in zip(vector_a, vector_b):
        
        # Garante que a operação use inteiros padrão antes da saturação
        a_val = np.int8(a_elem) 
        b_val = np.int8(b_elem)

        # 2. Execução da Operação
        if alu_op_sel == 0: # SOMA ('0' no VHDL)
            operation_result = int(a_val) + int(b_val)
        else: # SUBTRAÇÃO ('1' no VHDL)
            operation_result = int(a_val) - int(b_val)
        
        # 3. Aplicação da Saturação (Clipping)
        result_saturated = saturate_8bit(operation_result)
        result_vector.append(result_saturated)
        
    return result_vector, alu_op_sel

# --- Função de Execução e Timing ---

def run_test_case(test_id, vector_a, vector_b, op_sel, expected_e, test_desc):
    """Executa o teste, mede o tempo e exibe o resultado e verificação."""
    
    start_time = time.perf_counter()
    resultado_vector, op_code = alu_vector_op(vector_a, vector_b, op_sel)
    end_time = time.perf_counter()
    duration = (end_time - start_time) * 1e6 # µs
    
    op_char = '+' if op_code == 0 else '-'
    op_name = "SOMA" if op_code == 0 else "SUB"
    
    # 1. Verificação Elemento a Elemento
    passed = True
    for i in range(C_VECTOR_SIZE):
        expected_val = np.int8(expected_e[i])
        actual_val = resultado_vector[i]
        
        if actual_val != expected_val:
            passed = False
            break

    verification = '✅ PASSOU' if passed else '❌ FALHOU'
    
    print(f"\n--- Teste {test_id}: {op_name} ({test_desc}) ---")
    print(f"  Vetor A: {vector_a}")
    print(f"  Vetor B: {vector_b}")
    print(f"  Operação: {op_name}")
    print(f"  Resultado Saída: {list(map(int, resultado_vector))}")
    print(f"  Valor Esperado:  {expected_e}")
    print(f"  Tempo de Execução (Python): {duration:.3f} µs")
    
    if not passed:
        print("  Detalhes da Falha:")
        for i in range(C_VECTOR_SIZE):
            expected_val = expected_e[i]
            actual_val = resultado_vector[i]
            if actual_val != expected_val:
                print(f"    Elemento {i} ({vector_a[i]} {op_char} {vector_b[i]}): Obtido {int(actual_val)} | Esperado {expected_val}")
    
    print(f"  Verificação: {verification}")
    
    return resultado_vector

# --- Execução dos Casos de Teste (Alinhados ao VHDL) ---

print("## Testbench Somador/Subtrator Vetorial (ALU) - Alinhado ao VHDL ##")

# Teste 1: SOMA Normal (10 + 5 = 15)
run_test_case(
    1, 
    vector_a=[10, 20, 30, 40],
    vector_b=[5, 5, 5, 5],
    op_sel=0, # SOMA
    expected_e=[15, 25, 35, 45], 
    test_desc="SOMA Normal"
)
        
# Teste 2: SOMA com Overflow (Saturação em 127)
# Elementos críticos: 100+40=140, 126+5=131, 127+1=128. Todos saturam em 127.
run_test_case(
    2, 
    vector_a=[MAX_VAL, 100, 126, 127], 
    vector_b=[1, 40, 5, 1], 
    op_sel=0, # SOMA
    expected_e=[MAX_VAL, MAX_VAL, MAX_VAL, MAX_VAL], 
    test_desc="SOMA com Saturação Positiva"
)

# Teste 3: SUBTRAÇÃO Normal (50 - 10 = 40)
run_test_case(
    3, 
    vector_a=[50, 60, 70, 80], 
    vector_b=[10, 10, 10, 10], 
    op_sel=1, # SUBTRAÇÃO
    expected_e=[40, 50, 60, 70], 
    test_desc="SUB Normal"
)

# Teste 4: SUBTRAÇÃO com Underflow (Saturação em -128)
# Elementos críticos: -120-10=-130, -100-40=-140, -127-2=-129, -128-1=-129. Todos saturam em -128.
run_test_case(
    4, 
    vector_a=[-120, -100, -127, -128], 
    vector_b=[10, 40, 2, 1], 
    op_sel=1, # SUBTRAÇÃO
    expected_e=[MIN_VAL, MIN_VAL, MIN_VAL, MIN_VAL], 
    test_desc="SUB com Saturação Negativa"
)

## Testbench Somador/Subtrator Vetorial (ALU) - Alinhado ao VHDL ##

--- Teste 1: SOMA (SOMA Normal) ---
  Vetor A: [10, 20, 30, 40]
  Vetor B: [5, 5, 5, 5]
  Operação: SOMA
  Resultado Saída: [15, 25, 35, 45]
  Valor Esperado:  [15, 25, 35, 45]
  Tempo de Execução (Python): 18.000 µs
  Verificação: ✅ PASSOU

--- Teste 2: SOMA (SOMA com Saturação Positiva) ---
  Vetor A: [np.int8(127), 100, 126, 127]
  Vetor B: [1, 40, 5, 1]
  Operação: SOMA
  Resultado Saída: [127, 127, 127, 127]
  Valor Esperado:  [np.int8(127), np.int8(127), np.int8(127), np.int8(127)]
  Tempo de Execução (Python): 7.100 µs
  Verificação: ✅ PASSOU

--- Teste 3: SUB (SUB Normal) ---
  Vetor A: [50, 60, 70, 80]
  Vetor B: [10, 10, 10, 10]
  Operação: SUB
  Resultado Saída: [40, 50, 60, 70]
  Valor Esperado:  [40, 50, 60, 70]
  Tempo de Execução (Python): 7.200 µs
  Verificação: ✅ PASSOU

--- Teste 4: SUB (SUB com Saturação Negativa) ---
  Vetor A: [-120, -100, -127, -128]
  Vetor B: [10, 40, 2, 1]
  Operação: SUB
  Re

[np.int8(-128), np.int8(-128), np.int8(-128), np.int8(-128)]